In [1]:
from WikiSeriesNN import WikiSeriesNN

from WikiPlot import plot_prediction
from Seq2SeqConv import Seq2SeqConv
from Seq2SeqConvFull import Seq2SeqConvFull
from Seq2Seq import Seq2Seq
from os import path
import pickle
from PerformanceMetrics import smape, smape_batch

import numpy as np

import ipywidgets as widgets

Using TensorFlow backend.


In [2]:
root_folder = "./data"
train_file = path.join(root_folder,"train_meta.pkl")

In [3]:
series = WikiSeriesNN(train_file)

Data ranges from 2015-07-01 to 2017-09-10


In [4]:
saved_path = path.join(root_folder,"seqConv.h5")
seq2seqConv = Seq2SeqConv(series,60)
seq2seqConv.load_model(saved_path)

Train encoding: 2015-07-01 00:00:00 - 2017-05-13 00:00:00
Train prediction: 2017-05-14 00:00:00 - 2017-07-12 00:00:00 

Val encoding: 2015-08-30 00:00:00 - 2017-07-12 00:00:00
Val prediction: 2017-07-13 00:00:00 - 2017-09-10 00:00:00

Encoding interval: 683
Prediction interval: 60
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          96        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 32)          2080      
_________________________________________________________________
con

In [5]:
saved_path = path.join(root_folder,"seqConvFull.h5")
seq2seqConvFull = Seq2SeqConvFull(series,60)
seq2seqConvFull.load_model(saved_path)

Train encoding: 2015-07-01 00:00:00 - 2017-05-13 00:00:00
Train prediction: 2017-05-14 00:00:00 - 2017-07-12 00:00:00 

Val encoding: 2015-08-30 00:00:00 - 2017-07-12 00:00:00
Val prediction: 2017-07-13 00:00:00 - 2017-09-10 00:00:00

Encoding interval: 683
Prediction interval: 60
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 1)      0                                            
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, None, 16)     32          input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, None, 32)     1056        conv1d_12[0][0]                  
_________________________

In [9]:
%%time
samples_count = series.df.shape[0]
smape_seq2seq = 0.0
smape_conv = 0.0
smape_conv_full = 0.0

encode_all = np.zeros(shape=(samples_count,683,1))
target_all = np.zeros(shape=(60,samples_count))
i = 0

samples_indexes = np.arange(samples_count)

for index in samples_indexes:
    _,_,encode,target = seq2seqConvFull.series.get_sample(index=index)
    target_all[:,i] = target[:,0]
    encode_all[i] = encode
    i += 1

CPU times: user 20.6 s, sys: 0 ns, total: 20.6 s
Wall time: 20.6 s


In [10]:
%%time
predict_full = seq2seqConvFull.predictBatch(encode_all,target_all,False, batch=2**11)
print(predict_full.shape)

(60, 106328)
(106328, 683, 1)
(106328, 60, 1)
CPU times: user 13min 3s, sys: 6min 48s, total: 19min 51s
Wall time: 18min 31s


In [11]:
%%time
predict_conv = seq2seqConv.predictBatch(encode_all,target_all,False, batch=2**11)
print(predict_conv.shape)

(60, 106328)
(106328, 683, 1)
(106328, 60, 1)
CPU times: user 2min 45s, sys: 1min 18s, total: 4min 3s
Wall time: 4min


In [12]:
%%time
encode_conv,target_conv,prediction_conv = seq2seqConv.normalise_reshape_prediction_batches(encode_all,target_all,predict_conv,samples_indexes)
encode_full,target_full,prediction_full = seq2seqConvFull.normalise_reshape_prediction_batches(encode_all,target_all,predict_full,samples_indexes)

CPU times: user 3.38 s, sys: 572 ms, total: 3.96 s
Wall time: 3.95 s


In [14]:
folder_conv = path.join(root_folder,"conv")
folder_full = path.join(root_folder,"full")

In [16]:
np.save(path.join(folder_conv,"encode"),encode_conv)
np.save(path.join(folder_conv,"target"),target_conv)
np.save(path.join(folder_conv,"prediction"),prediction_conv)

In [17]:
np.save(path.join(folder_full,"encode"),encode_full)
np.save(path.join(folder_full,"target"),target_full)
np.save(path.join(folder_full,"prediction"),prediction_full)